In [1619]:
from datetime import datetime, timedelta
from ordered_set import OrderedSet
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from utils import inward_velocity_vector, random_edge_position, generate_timestamps
import random 
import numpy as np


q = 0.05
num_steps = 500
FOV = [0,500]

birth_rate = 0.015
death_probability = 0
time_stepsize = 1
start_time_ = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
start_time = datetime.strptime(start_time_, '%Y-%m-%dT%H:%M:%S')
timestep_size = timedelta(seconds=1)
dt = 1
times = generate_timestamps(start_time_, dt, num_steps)

car_ratio = 0.7

truths = OrderedSet()  # Truths across all time
current_truths = set() # Truths alive at the current time

# Define a transition model with some noise
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q)])

# Calculate the FOV center
fov_center = (FOV[1]-FOV[0])/2

timesteps = []
for k in range(num_steps):
    timesteps.append(start_time + timedelta(seconds=k))

    # Death (Only for truths within FOV)
    for truth in current_truths.copy():
        if truth[-1].state_vector[0] < FOV[0] or truth[-1].state_vector[0] > FOV[1]: 
            current_truths.remove(truth)

    # Update truths
    for truth in current_truths:
        truth.append(GroundTruthState(transition_model.function(truth[-1], noise=True, time_interval=timedelta(seconds=1)), timestamp=timesteps[k]))

    # Birth based on Poisson process
    num_births = np.random.poisson(birth_rate)
    
    for _ in range(num_births):
        vehicle_type = 'car' if np.random.rand() < car_ratio else 'train'
        initial_position = random_edge_position(FOV)
        initial_velocity = 5 if initial_position == FOV[0] else -5
        state = GroundTruthState([initial_position, initial_velocity], timestamp=timesteps[k], metadata={'vehicle_type' : vehicle_type})
        truth = GroundTruthPath([state])
        current_truths.add(truth)
        truths.add(truth)

In [1620]:
# Amplitude distributions for clutter, car and train
from scipy.stats import norm
pi_0_1 = 0.5
pi_0_2 = 0.5

alpha_0 = -10
alpha_1 = -8.8
alpha_2 = -7

r_0 = 0.2
r_1 = 1
r_2 = 1.2

phi_0 = norm(loc=alpha_0, scale= r_0)
phi_1 = norm(loc=alpha_1, scale= r_1)
phi_2 = norm(loc=alpha_2, scale= r_2)

In [1621]:
from scipy.stats import uniform
from stonesoup.types.detection import TrueDetection
from stonesoup.types.detection import Clutter
from stonesoup.models.measurement.linear import LinearGaussian


global_clutter_rate = 2
clutter_area = FOV
r = 0.05
prob_detect = 0.90
clutter_spatial_density = global_clutter_rate / (FOV[1]-FOV[0])


# Convert OrderedSet to a list for shuffling
truths_list = list(truths)


np.random.shuffle(truths_list)


measurement_model = LinearGaussian(
    ndim_state=2,
    mapping=(0,),
    noise_covar=np.array([[r]])
    )
all_measurements = []

for k in range(num_steps):
    measurement_set = set()
    timestamp = start_time + timedelta(seconds=k)

    # Generate general background clutter in all of FOV
    num_clutter = np.random.poisson(global_clutter_rate)
    phi_0_samples = phi_0.rvs(size=num_clutter)
    for i in range(num_clutter):
        x = uniform.rvs(FOV[0], FOV[1])
        measurement_set.add(Clutter(np.array([[x]]), timestamp=timestamp, measurement_model=measurement_model, metadata={'amplitude': phi_0_samples[i]}))

    # Loop through each truth in the shuffled list
    for idx, truth in enumerate(truths_list):
        try:
            truth_state = truth[timestamp]
        except IndexError:
            # This truth not alive at this time.
            continue
        
        
        # Sample the amplitude based on the type of truth
        if truth[0].metadata['vehicle_type'] == 'car':
            amplitude_samples = phi_1.rvs(size=1)  # Sampling from phi_1 for cars
        else:
            amplitude_samples = phi_2.rvs(size=1)  # Sampling from phi_2 for trains
        
        # Generate actual detection from the state with a 10% chance that no detection is received.
        if np.random.rand() <= prob_detect:
            # Generate actual detection from the state
            measurement = measurement_model.function(truth_state, noise=True)
            
            # Add the sampled amplitude as metadata depending on whether it's a car or train
            metadata_value = amplitude_samples[0]  # Assuming a single value from the distribution
            
            measurement_set.add(TrueDetection(state_vector=measurement,
                                              groundtruth_path=truth,
                                              timestamp=truth_state.timestamp,
                                              measurement_model=measurement_model,
                                              metadata={'amplitude': metadata_value}))


        

    all_measurements.append(measurement_set)

In [1622]:
from utils import SimPlotter
plotter = SimPlotter(truths, all_measurements, FOV = FOV)

In [1623]:
plotter.plot_measurements(plotly=True)

In [1624]:
from CustomDeleter import CustomDeleter
deleter = CustomDeleter(covar_trace_thresh=30, fov = FOV)

from stonesoup.predictor.kalman import KalmanPredictor
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(2)])
predictor = KalmanPredictor(transition_model)

from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
hypothesiser_initiator = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)

from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator_initiator = GNNWith2DAssignment(hypothesiser_initiator)

from stonesoup.types.state import GaussianState
from CustomInitiator import RestrictedInitiator

initiator = RestrictedInitiator(
    prior_state = None,
    measurement_model=measurement_model,
    deleter=deleter,
    data_associator=data_associator_initiator,
    updater=updater,
    min_points=4,
    FOV = FOV,
    range = 30
    )

from stonesoup.hypothesiser.probability import PDAHypothesiser
from stonesoup.dataassociator.probability import JPDA
from tracker import JPDAtracker
hypothesiser = PDAHypothesiser(
    predictor=predictor,
    updater=updater,
    clutter_spatial_density=clutter_spatial_density,
    prob_detect=prob_detect
 
)

data_associator = JPDA(hypothesiser=hypothesiser)



tracker = JPDAtracker(
    updater = updater,
    data_associator = data_associator,
    initiator = initiator,
    deleter = deleter
)

In [1625]:
# Create empirical likelihood models based on a few "logged""heatmaps and picks"
amp_1 = phi_1.rvs(size=10)
amp_2 = phi_2.rvs(size=3)

phi__1 = norm(loc=np.mean(amp_1), scale=np.std(amp_1))
phi__2 = norm(loc=np.mean(amp_2), scale=np.std(amp_2))


In [1634]:
print(np.mean(amp_1), np.std(amp_1))
print(np.mean(amp_2), np.std(amp_2))

-8.812608295128472 1.3910891985724876
-8.20871816469498 1.2444924775489918


In [1626]:
all_tracks = tracker.track(all_measurements,times[0],dt=dt, phi_1=phi__1, phi_2=phi__2, pi_0_1=pi_0_1, pi_0_2=pi_0_2)

In [1627]:
from utils import create_track_dataframe
from colors import named_colors
import random
df = create_track_dataframe(all_tracks, tracker, update_thres=10)



In [1628]:
from utils import create_track_dataframe
from colors import named_colors
import random


random.seed(42)
colors = random.sample(named_colors, df['vehicle index'].nunique())

measurements = [measurement.state_vector[0] for measurement in [measurement for measurement_set in all_measurements for measurement in measurement_set]]
measurement_timestamps = [measurement.timestamp for measurement in [measurement for measurement_set in all_measurements for measurement in measurement_set]]
fig = go.Figure(layout=dict(width=800, height=600))
fig.add_trace(go.Scatter(x=measurements, y=measurement_timestamps, mode='markers', marker=dict(symbol='x', color = 'green'), name='Measurements'))
fig.add_shape(type="line",
                    x0=FOV[0],
                    y0=min(measurement_timestamps),
                    x1=FOV[0],
                    y1=max(measurement_timestamps),
                    line=dict(color="red", width=2)
                )

fig.add_shape(
                    type="line",
                    x0=FOV[1],
                    y0=min(measurement_timestamps),
                    x1=FOV[1],
                    y1=max(measurement_timestamps),
                    line=dict(color="red", width=2)
                )
                
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines', marker=dict(color='red'), name='Field of View'))

vel_traces = []
lower_traces = []
upper_traces = []

for vehicle_idx, group_df in df.groupby('vehicle index'):
    
    color = colors[vehicle_idx]
    vel_trace = go.Scatter(
        x=group_df['position [m]'],  
        y=group_df['timestamp'],
        mode='lines+markers',
        line=dict(color=color, width=2),
        name=f'Track {vehicle_idx+1}',
        showlegend = True 
    )
    lower = go.Scatter(
        x=group_df['pos lower'],  
        y=group_df['timestamp'],
        mode='lines',
        line=dict(color=color, width=1, dash='dash'),
        name=f'Track {vehicle_idx+1}',
        showlegend = False
    )
    upper = go.Scatter(
        x=group_df['pos upper'],  
        y=group_df['timestamp'],
        mode='lines',
        line=dict(color=color, width=1, dash='dash'),
        name=f'Track {vehicle_idx+1}',
        showlegend = False
    )
    fig.add_trace(vel_trace)
    fig.add_trace(lower)
    fig.add_trace(upper)


fig.update_layout(
    title=f"Measurements and estimated tracks",
    xaxis_title="Position",
    yaxis_title="Time [s]",
    autosize=False,
    width=1000,
    height=800,
)

In [1629]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Assuming your DataFrame is named 'df'
df['is_nan'] = np.isnan(df['car probability'])



# Create an empty figure
fig = go.Figure()
fig.update_layout(width=800, height=600)

# Iterate through each vehicle index
for vehicle_idx, group_df in df.groupby('vehicle index'):
    # Scatter plot for non-NaN values within each vehicle index
    scatter_non_nan = go.Scatter(
        x=group_df.loc[~group_df['is_nan'], 'position [m]'],
        y=group_df.loc[~group_df['is_nan'], 'timestamp'],
        mode='markers+lines',
        marker=dict(
            size=6,
            color=group_df.loc[~group_df['is_nan'], 'car probability'],
            colorscale='RdYlBu',
            cmin=0,  # Set the minimum value for the color scale
            cmax=1,  # Set the maximum value for the color scale
            colorbar=dict(title='Car Probability'),
        ),
        line=dict(color='rgba(0,0,0,0.3)'),  # Add a line connecting points
        hovertext=group_df.loc[~group_df['is_nan'], 'track ID'],
        name=f"Vehicle Index {vehicle_idx}",  # Specify a name for the legend
        showlegend=False
    )
    
    # Add the trace to the figure
    fig.add_trace(scatter_non_nan)

    # Scatter plot for NaN values within each vehicle index
    scatter_nan = go.Scatter(
        x=group_df.loc[group_df['is_nan'], 'position [m]'],
        y=group_df.loc[group_df['is_nan'], 'timestamp'],
        mode='markers+lines',
        marker=dict(
            size=6,
            color='white',
        ),
        hovertext=group_df.loc[group_df['is_nan'], 'track ID'],
        showlegend=False,  # Hide legend for NaN values
    )
    
    # Add the trace to the figure
    fig.add_trace(scatter_nan)

# Set layout customization
fig.update_layout(
    title='Track Positions with Car Probability',
    xaxis=dict(title='Position'),
    yaxis=dict(title='Time [s]'),
    showlegend=True,
)

# Show the plot
fig.show()


In [1632]:
for truth in truths:
    print(truth[0].timestamp,truth[0].metadata['vehicle_type'])

2024-01-08 13:15:22 train
2024-01-08 13:15:55 car
2024-01-08 13:17:04 train
2024-01-08 13:17:31 car
2024-01-08 13:18:38 car
2024-01-08 13:18:54 train
2024-01-08 13:19:01 train
2024-01-08 13:19:20 train
2024-01-08 13:19:26 train
2024-01-08 13:19:29 car
2024-01-08 13:19:32 car
